In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

packages = [
    'org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1',
    'org.apache.kafka:kafka-clients:2.8.1'
]

# SparkSession 초기화
spark = SparkSession.builder \
    .master("yarn") \
    .appName("apiUsageKafkaConsumer") \
    .config("spark.submit.deployMode", "client") \
    .config("spark.jars.packages", ",".join(packages)) \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

# Kafka value 에 있는 json구조
api_usage_trace_schema = StructType([
    StructField("eventId", StringType(), False),
    StructField("requestTime", TimestampType(), False),
    StructField("responseTime", TimestampType(), False),
    StructField("requestProtocol", StringType(), True),
    StructField("requestMethod", StringType(), True),
    StructField("requestUri", StringType(), False),
    StructField("responseStatus", IntegerType(), True),
    StructField("clientIp", StringType(), True),
    StructField("clientAgent", StringType(), True),
    StructField("apiKey", StringType(), True),
    StructField("traceId", StringType(), False)
])

# 각종 설정 변수
kafka_topic = "api-usage-trace" # 카프카 토픽
kafka_servers = "Kafka00Container:9092" 
offset_reset = "earliest"
time_zone = "Asia/Seoul" # 나의 경우, 한국 기준으로 date를 뽑아내야해서

# Kafka에서 가져오는 df
df = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", kafka_servers) \
    .option("subscribe", kafka_topic) \
    .option("includeHeaders", "true") \
    .option("startingOffsets", offset_reset) \
    .load()

transformed_df = df.select(
    from_json(col("value").cast("string"), api_usage_trace_schema).alias("parsed_value"), # json 파싱
    col("offset"),
    col("partition"),
    col("timestamp")
).select(
    col("parsed_value.*"), # flat하게
    col("offset").alias("kafka_offset"),
    col("partition").alias("kafka_partition"),
    col("timestamp").alias("kafka_timestamp")
).withColumn(
    "date", to_date(from_utc_timestamp(col("requestTime"), time_zone))
)

# Date별로 파티셔닝 해서, parquet으로 저장
query = transformed_df \
    .writeStream \
    .format("parquet") \
    .outputMode("append") \
    .partitionBy("date") \
    .trigger(processingTime='5 seconds') \
    .option("path", "/tracking/api_usage") \
    .option("checkpointLocation", "/tracking/api_usage/check_point") \
    .start()

query.awaitTermination()